In [63]:
import torch
from torch import nn
import numpy as np
import tinycudann as tcnn
import json

### Pay Attention to:
# 1. The hidden layers of MLP and TCNN are different (MLP has one more hidden layer than TCNN)
# 2. TCNN mlp has no bias

class MLP(nn.Module):
    def __init__(self, in_dims, hidden_dims, out_dims, hidden_layers):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dims, hidden_dims, bias=False),
            nn.ReLU(inplace=True),
            *[nn.Sequential(
                nn.Linear(hidden_dims, hidden_dims, bias=False),
                nn.ReLU(inplace=True)
            ) for _ in range(hidden_layers)],
            nn.Linear(hidden_dims, out_dims, bias=False)
        )
        
        # randomly init the weights and bias
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                # nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.net(x)
    
class TCNN(nn.Module):
    def __init__(self, in_dims, hidden_dims, out_dims, hidden_layers):
        super().__init__()
        
        self.net = tcnn.Network(
            n_input_dims=in_dims,
            n_output_dims=out_dims,
            network_config={
                "otype": "CutlassMLP",
                "activation": "ReLU",
                "output_activation": "None",
                "n_neurons": hidden_dims,
                "n_hidden_layers": hidden_layers + 1,
            },
        )
    
    def forward(self, x):
        return self.net(x)

In [67]:
def save_mlp_as_tcnn_json(mlp_model, save_path, hidden_dims, out_dims, dims_multiplier=16):
    mlp_model_keys = mlp_model.state_dict().keys()

    weights = []
    for key in mlp_model_keys:
        weights.append(mlp_model.state_dict()[key].cpu().numpy().flatten())

    # padding the last layer output dims to 32
    padded_dims = (out_dims // dims_multiplier + 1) * dims_multiplier
    weights[-1] = np.pad(weights[-1], (0, hidden_dims*padded_dims - weights[-1].flatten().shape[0]), 'constant', constant_values=0)

    # concatenate the weights
    weights = np.concatenate(weights)

    # save the weights to json
    json_weights = {}
    params_type = 'float'
    params_binary = {}
    
    bytes = weights.astype(np.float32).tobytes()
    params_binary['bytes'] = [byte for byte in bytes]
    params_binary['subtype'] = None
    
    json_weights['n_params'] = weights.shape[0]
    json_weights['params_type'] = params_type
    json_weights['params_binary'] = params_binary
    
    with open(save_path, 'w') as f:
        json.dump(json_weights, f)
    
        
def tcnn_read_json(json_path, tcnn_model):
    with open(json_path, 'r') as f:
        json_weights = json.load(f)
    
    params_binary = json_weights['params_binary']['bytes']
    params_fp32 = np.frombuffer(bytes(params_binary), dtype=np.float32)
    
    # load the weights to tcnn model
    checkpoint = {
        'net.params': torch.from_numpy(params_fp32)
    }
    
    # load the weights to tcnn model
    # pay attention that tcnn_model is in device or host
    tcnn_model.load_state_dict(checkpoint)
    
    return tcnn_model

In [65]:
in_dims = 32
hidden_dims = 64
out_dims = 25
hidden_layers = 3

save_path = "tcnn_mlp.json"

mlp_model = MLP(in_dims, hidden_dims, out_dims, hidden_layers)
tcnn_model = TCNN(in_dims, hidden_dims, out_dims, hidden_layers)

# test the tcnn model
x = torch.randn(1, 32)
mlp_output = mlp_model(x)

In [66]:
save_mlp_as_tcnn_json(mlp_model, save_path, hidden_dims, out_dims)
tcnn_read_json(save_path, tcnn_model)

tcnn_output = tcnn_model(x)

print("MLP output: ", mlp_output)
print("TCNN output: ", tcnn_output)

MLP output:  tensor([[ 1.0674, -0.8858, -1.5892, -3.3803,  1.0649,  0.7968, -2.2129,  2.9356,
         -0.9036, -0.5457,  0.5316, -0.2651, -0.8903, -2.4988, -1.2052,  0.9512,
         -2.3698,  0.4381,  2.8194, -0.5864,  0.3877, -3.1824,  0.7544,  0.2227,
          0.8431]], grad_fn=<MmBackward0>)
TCNN output:  tensor([[ 1.0674, -0.8853, -1.5889, -3.3848,  1.0635,  0.7949, -2.2109,  2.9336,
         -0.9023, -0.5498,  0.5303, -0.2610, -0.8872, -2.4961, -1.2061,  0.9531,
         -2.3711,  0.4392,  2.8164, -0.5869,  0.3862, -3.1797,  0.7495,  0.2219,
          0.8472]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SliceBackward0>)


C:\Users\Yijie Deng\AppData\Local\Temp\ipykernel_16136\3818351087.py:41: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  'net.params': torch.from_numpy(params_fp32)
e:\anaconda3\envs\main\lib\site-packages\tinycudann\modules.py:178: UserWarning: input must be a CUDA tensor, but isn't. This indicates suboptimal performance.
  warnings.warn("input must be a CUDA tensor, but isn't. This indicates suboptimal performance.")


In [68]:
save_mlp_as_tcnn_json(mlp_model, save_path, hidden_dims, out_dims)
tcnn_read_json(save_path, tcnn_model)

tcnn_output = tcnn_model(x)

print("MLP output: ", mlp_output)
print("TCNN output: ", tcnn_output)

MLP output:  tensor([[ 1.0674, -0.8858, -1.5892, -3.3803,  1.0649,  0.7968, -2.2129,  2.9356,
         -0.9036, -0.5457,  0.5316, -0.2651, -0.8903, -2.4988, -1.2052,  0.9512,
         -2.3698,  0.4381,  2.8194, -0.5864,  0.3877, -3.1824,  0.7544,  0.2227,
          0.8431]], grad_fn=<MmBackward0>)
TCNN output:  tensor([[ 1.0674, -0.8853, -1.5889, -3.3848,  1.0635,  0.7949, -2.2109,  2.9336,
         -0.9023, -0.5498,  0.5303, -0.2610, -0.8872, -2.4961, -1.2061,  0.9531,
         -2.3711,  0.4392,  2.8164, -0.5869,  0.3862, -3.1797,  0.7495,  0.2219,
          0.8472]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SliceBackward0>)


In [1]:
32 * 64 + 64 * 64 * 2 + 64 * 32

12288